In [113]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [114]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
  
#brown corpora
corpus=brown.words()

#calculate frequency of words
word_freq = nltk.FreqDist(w.lower() for w in corpus)


bigram_corpus=[]
previous='#empty'

#recreate corpus
for word in corpus:
  if previous in ['#empty','.','?','!']:
    bigram_corpus.append('#start_end#')
  if word_freq[word]==1:
    bigram_corpus.append('#oov#')
  else:
    bigram_corpus.append(word)
  previous=word

#update frequency
new_word_freq = nltk.FreqDist(w.lower() for w in bigram_corpus)

#bigrams corpora
bigrams = nltk.bigrams(w.lower() for w in bigram_corpus)
cfd = nltk.ConditionalFreqDist(bigrams)
total_words=len(bigram_corpus)

#define function unigram probability
def unigram_probability(word):
  if word in bigram_corpus:
    return new_word_freq[word]/total_words#prob=freq/total words count
  else:
    return new_word_freq['#oov#']/total_words

#define function bigram probability
def bigram_probability(word2,word1):
  if word2 not in cfd[word1]:
    return unigram_probability(word1)
  else:
    bigram_frequency = cfd[word1][word2]#prob=freq(n,n-1)/freq(n-1)
    unigram_frequency = new_word_freq[word1]
    return (bigram_frequency/unigram_frequency)

#define function calculate probabilty
def calculate_probability(sentence):
  x=word_tokenize(sentence)#tokenize sentences
  x.insert(0,"#start_end#")
  x.append("#start_end#")
  
  for i in range(len(x)):
    if x[i] not in bigram_corpus:#replace out of vocabulary words with #oov#
      x[i]="#oov#"
    x[i]=x[i].lower()#lower words
  print("Tokenized sentence:",x)
  y=[]
  for i in range(1,len(x)):
    y.append((x[i],x[i-1]))#create bi-grams
  print("Bigrams:",y)
  proba=1
  for word in y:
    res=bigram_probability(word[1],word[0])
    print("(",word[1],",",word[0],"):",res)
    proba=proba*res
  
  return proba

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [116]:
sentence=input("Enter sentence: ")
result=calculate_probability(sentence)
print("Probability of this sentence using bigram model is:",result)

Enter sentence: this is one of the best experience.
Tokenized sentence: ['#start_end#', 'this', 'is', 'one', 'of', 'the', 'best', 'experience', '.', '#start_end#']
Bigrams: [('this', '#start_end#'), ('is', 'this'), ('one', 'is'), ('of', 'one'), ('the', 'of'), ('best', 'the'), ('experience', 'best'), ('.', 'experience'), ('#start_end#', '.')]
( #start_end# , this ): 0.004228209926308341
( this , is ): 0.004352557127312296
( is , one ): 0.013669501822600244
( one , of ): 0.0026914204108535646
( of , the ): 0.057502833188283954
( the , best ): 0.008547008547008548
( best , experience ): 0.00022681942461829003
( experience , . ): 0.0405530120551237
( . , #start_end# ): 0.045721372060284655
Probability of this sentence using bigram model is: 1.3994624416756706e-19
